## Create Edits that match post edit class distribution of target class to real edits

In [1]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from datetime import datetime

In [2]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, list_to_dict, ensure_dir, informal_log, write_lists, get_common_dir_path
from utils.df_utils import load_and_preprocess_csv
from utils.visualizations import histogram
from utils.model_utils import prepare_device, quick_predict
from parse_config import ConfigParser
from test import predict, predict_with_bump
import datasets.datasets as module_data
import model.model as module_arch
import model.metric as module_metric
import model.loss as module_loss
from trainer.editor import Editor

In [3]:
# Define constants, paths
class_list_path = os.path.join('metadata', 'cinic-10', 'class_names.txt')

# config_path = 'configs/copies/edit_experiments/cinic10_imagenet_bump_noise.json'
config_path = 'configs/copies/edit_experiments/cinic10_imagenet_bump_corresponding.json'

In [5]:
# Load config file, models, and dataloader
class_list = read_lists(class_list_path)
class_idx_dict = list_to_dict(class_list)

config_dict = read_json(config_path)
config = ConfigParser(config_dict)
device, device_ids = prepare_device(config_dict['n_gpu'])

In [6]:
# Load model
layernum = config.config['layernum']
model = config.init_obj('arch', module_arch, layernum=layernum, device=device)
model.eval()


CIFAR10PretrainedModelEdit(
  (model): VGG(
    (normalize): InputNormalize()
    (layer0): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (layer1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (layer3): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, moment

In [7]:
# Load datasets
# config = ConfigParser(config_dict)
data_loader_args = dict(config.config["data_loader"]["args"])
dataset_args = dict(config["dataset_args"])

val_image_paths = read_lists(config_dict['dataset_paths']['valid_images'])
val_labels = read_lists(config_dict['dataset_paths']['valid_labels'])
val_paths_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=val_image_paths,
        labels=val_labels,
        return_paths=True,
        **dataset_args
    ),
    **data_loader_args
)

# Obtain loss function and metric functions
loss_fn = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

In [8]:
target_class_name = 'airplane'
target_class_idx = class_idx_dict[target_class_name]
n_select = 100
timestamp = '0209_112038'
bumps_timestamp = '0208_112555'
save_timestamp = '0213_143741'

root_dir = os.path.join('saved', 'edit', 'trials', 'CINIC10_ImageNet-VGG_16', '{}' + '_{}'.format(n_select), timestamp)
csv_path_template = os.path.join(root_dir, 'results_table.csv')


In [33]:
# def save_class_distribution(csv_path, 
#                             target_class_idx,
#                             target_class_name,
#                             show=False,
#                             histogram_save_path=None,
#                             data_save_path=None):
    
#     df = load_and_preprocess_csv(
#         csv_path=csv_path,
#         drop_duplicates=['ID']
#     )
    
#     # Obtain number of predictions for target class pre edit
#     pre_edit_class_distribution = df['Pre Class Dist'].to_numpy()
#     pre_edit_class_distribution = np.stack(pre_edit_class_distribution)
#     target_pre_edit_class_predictions = np.mean(pre_edit_class_distribution[:, target_class_idx])
    
#     # Obtain number of predictions for target class post edit for each trial
#     class_distribution = df['Post Class Dist'].to_numpy()
#     class_distribution = np.stack(class_distribution, axis=0)
#     target_class_distribution = class_distribution[:, target_class_idx]
#     # target_class_bins = np.bincount(target_class_distribution)
#     if histogram_save_path is None:
#         histogram_save_path = os.path.join(os.path.dirname(csv_path), 'graphs', 'summary', 'target_class_distribution.png')
#     title = 'Post Edit {} Class Distribution for {} Edits'.format(target_class_name, target_class_name)
#     xlabel = 'Num. {} Predictions Post Edit'.format(target_class_name)
#     ylabel = 'Num. Edits'
    
#     bin_values, bins, _ = histogram(
#         data=target_class_distribution,
#         n_bins=50,
#         title=title,
#         xlabel=xlabel,
#         ylabel=ylabel,
#         marker=target_pre_edit_class_predictions,
#         show=show,
#         save_path=histogram_save_path)
    
#     bin_tuples = []
#     for bin_idx in range(len(bins) - 1):
#         bin_tuples.append((bins[bin_idx], bins[bin_idx+1]))

#     save_data = {
#         "n_target_predictions": target_class_distribution,
#         "histogram_bin_values": bin_values,
#         "histogram_bins": bins
#     }
    
#     if data_save_path is None:
#         data_save_path = os.path.join(os.path.dirname(csv_path), 'target_class_distribution.pth')
#         torch.save(save_data, data_save_path)
    
#     print("Saved target class distribution & histogram data to {}".format(data_save_path))
    
#     plt.close('all')
#     return save_data

#### Loop over all classes and save histograms and distributions

In [ ]:
for class_name in class_list:
    csv_path = csv_path_template.format(class_name)
    save_class_distribution(
        csv_path=csv_path,
        target_class_idx=class_idx_dict[class_name],
        target_class_name=class_name,
        show=False)
    

In [10]:
# target_class_distribution_path = os.path.join(root_dir, 'target_class_distribution.pth').format(
#     target_class_name)
# target_class_distribution = torch.load(target_class_distribution_path)
# bin_lows = target_class_distribution['histogram_bins']

# bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
# ensure_dir(bump_save_dir)
# # Save a copy of histogram info to save_dir
# torch.save(target_class_distribution, os.path.join(bump_save_dir, 'target_class_distribution.pth'))
# # Run bump experiments
# print("Obtaining class distribution for {} from {}".format(target_class_name, target_class_distribution_path))
# print("Saving results to {}".format(bump_save_dir))
# match_bump_edits(
#     data_loader=val_paths_data_loader,
#     model=model,
#     loss_fn=loss_fn,
#     metric_fns=metric_fns,
#     device=device,
#     bin_lows=bin_lows,
#     target_class_idx=target_class_idx,
#     save_dir=bump_save_dir)

Obtaining class distribution for airplane from saved/edit/trials/CINIC10_ImageNet-VGG_16/airplane_100/0127_103716/target_class_distribution.pth
Saving results to saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100


 26%|██████████████████▍                                                   | 72/274 [00:32<01:30,  2.23it/s]


KeyboardInterrupt: 

In [ ]:
# bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
# metrics_save_path = os.path.join(bump_save_dir, 'bumps_preds_metrics.pth')

# bumped_target_class_dist = torch.load(metrics_save_path)

# bumped_hist_data = []


# target_class_dist_dict = torch.load(target_class_distribution_path)

# for n_target_predictions, bucket_value in zip(
#     bumped_target_class_dist['target_class_predictions'], 
#     target_class_distribution['histogram_bin_values']):
#     cur_data = [n_target_predictions for i in range(int(bucket_value))]
#     bumped_hist_data += cur_data
    
# bins = target_class_distribution['histogram_bins']
# bin_values = target_class_distribution['histogram_bin_values']

# histogram_save_path = os.path.join(
#     bump_save_dir, 
#     'graphs',
#     'summary',
#     'bumped_target_class_distribution.png')
# bump_bin_values, bump_bins, _= histogram(
#         data=bumped_hist_data,
#         n_bins=bins, #50,
#         title='Bumped Post Edit {} Class Distribution to Match {} Edits'.format(target_class_name, target_class_name),
#         xlabel='Num. {} Predictions Post Bump'.format(target_class_name),
#         ylabel='Num. Edits',
#         save_path=histogram_save_path,
#         show=True)

# assert (bin_values == bump_bin_values).all()

### Perform Bump for a Corresponding Edit

In [9]:
def match_bump(n_target_predictions,
               target_class_idx,
               bumps_preds_metrics,
               results_save_dir,
               data_loader,
               model,
               loss_fn,
               metric_fns,
               device,
               cushion=5,
               debug=True):
    '''
    Given a number of predictions for target class, obtain bump amount to match and save post edit metrics in results_save_dir
    
    Arg(s):
        n_target_predictions : int
            Number of predictions to obtain for target class
        target_class_idx : int
            index of target class
        bumps_preds_metrics : dict
            saved data from match_bump_edits()
        results_save_dir : str
            directory to save results to
        data_loader : torch.utils.data.DataLoader
            validation data loader to obtain metrics for
        model : torch.nn.Module
            model
        loss_fn : module
            loss function
        metric_fns : list[model.metric modules]
            list of metric functions
        device : torch.device
            GPU device to run model on
        cushion : int
            how far away cur_n_target_predictions can be from n_target_predictions on either side to break loop (buffer)
        debug : bool
            control verbosity
        
            
    Returns: 
        None
    '''
    # Unpack bumps_preds_metrics
    pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    target_class_predictions = bumps_preds_metrics['target_class_predictions']
    bump_amounts = bumps_preds_metrics['bump_amounts']
    
    # Ensure that the n_target_predictions > pre edit # predictions of target class
    pre_edit_n_target_predictions = pre_edit_metrics['predicted_class_distribution'][target_class_idx]
    assert pre_edit_n_target_predictions < n_target_predictions, \
        "n_target_predictions ({}) must be greater than pre-edit n_target_predictions ({})".format(n_target_predictions, pre_edit_n_target_predictions) 
    n_predictions_total = np.sum(pre_edit_metrics['predicted_class_distribution'])
    assert n_target_predictions <= n_predictions_total, \
        "n_target_predictions ({}) must be less than total number of data in dataloader ({})".format(n_target_predictions, n_predictions_total)
    # Find index above and below n_target_predictions
    bin_high_idx = -1
    for bin_idx, target_class_prediction in enumerate(target_class_predictions):
        if target_class_prediction > n_target_predictions:
            bin_high_idx = bin_idx
            break
    if bin_high_idx == -1: # Past upper end 
        # n_predictions_upper_bound
        bump_amount_upper_bound = bump_amounts[bin_high_idx] * 2
        bump_amount_lower_bound = bump_amounts[bin_high_idx]
        # raise ValueError("Out of bounds bin index")
        # return
    else: 
        n_predictions_upper_bound = target_class_predictions[bin_high_idx]
        bump_amount_upper_bound = bump_amounts[bin_high_idx]

        # Store lower bounds for bump_amount and n_predictions
        bin_low_idx = bin_high_idx - 1
        if bin_low_idx > -1: # First bin is already higher than n_target_predictions
            n_predictions_lower_bound = target_class_predictions[bin_low_idx]
            bump_amount_lower_bound = bump_amounts[bin_low_idx]
        else:
            n_predictions_lower_bound = 0
            bump_amount_lower_bound = 0
        
    # check to make sure that the targe
    # cur_bump_amount = (bump_amount_lower_bound + bump_amount_upper_bound) / 2
    cur_n_target_predictions = 0
    
    if debug:
        print("target n_predictions: {}".format(n_target_predictions))
        print("Initial bounds for bump: ({}, {})".format(bump_amount_lower_bound, bump_amount_upper_bound))
    while abs(cur_n_target_predictions - n_target_predictions) > cushion:
        cur_bump_amount = (bump_amount_lower_bound + bump_amount_upper_bound) / 2
        log = predict_with_bump(
                data_loader=data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                target_class_idx=target_class_idx,
                bump_amount=cur_bump_amount,
                output_save_path=os.path.join(results_save_dir, "post_edit_logits.pth"),
                log_save_path=os.path.join(results_save_dir, "post_edit_metrics.pth"))

        # Obtain num. predictions for target class and determine bin idx
        post_class_distribution = log['predicted_class_distribution']
        cur_n_target_predictions = post_class_distribution[target_class_idx]
        if debug:
            print("cur_bump_amount: {}, cur_n_target_predictions: {}".format(cur_bump_amount, cur_n_target_predictions))
        if cur_n_target_predictions > n_target_predictions:
            bump_amount_upper_bound = cur_bump_amount
            if debug:
                print("Updated upper bound to {}".format(bump_amount_upper_bound))
        elif cur_n_target_predictions < n_target_predictions:
            bump_amount_lower_bound = cur_bump_amount
            if debug:
                print("Updated lower bound to {}".format(bump_amount_lower_bound))
        
    if debug:
        print("final results: bump amount: {} n_target_predictions: {}".format(cur_bump_amount, cur_n_target_predictions))
    # if results_save_dir is not None:
    #     torch.save(log, os.path.join(results_save_dir, "post_edit_metrics.pth"))
        # torch.save(logits, os.path.join(results_save_dir, "post_edit_logits.pth"))
    return cur_bump_amount, cur_n_target_predictions, log
    

In [ ]:
# # testing

# bumps_preds_metrics_path = os.path.join(
#     "saved/edit/experiments/bump_edits",
#     "bumps_preds_metrics",
#     "{}_{}_bumps_preds_metrics.pth".format(target_class_name, n_select))
# bumps_preds_metrics = torch.load(bumps_preds_metrics_path)

# n_target_predictions = 65000
# results_save_dir = os.path.join('temp')
# match_bump(n_target_predictions=n_target_predictions,
#            target_class_idx=target_class_idx,
#            bumps_preds_metrics=bumps_preds_metrics,
#            results_save_dir=results_save_dir,
#            data_loader=val_paths_data_loader,
#            model=model,
#            loss_fn=loss_fn,
#            metric_fns=metric_fns,
#            device=device,
#            cushion=5,
#            debug=True)

In [21]:
n_edits = 5
cushion = 10
original_trials_trial_paths_path = os.path.join(root_dir.format(target_class_name), 'trial_paths.txt')
original_trial_paths = read_lists(original_trials_trial_paths_path)
common_path = get_common_dir_path(original_trial_paths)

# Data structure to store how much to bump for each n_predictions in target class

bump_amount_dictionary_path = os.path.join(
    'metadata',
    config_dict['name'], # CINIC10_ImageNet-VGG16
    'bump_amounts',
    '{}_{}'.format(target_class_name, n_select),
    'logit_bump_buffer_{}_dict.pth'.format(cushion))

if os.path.isfile(bump_amount_dictionary_path):
    bump_amount_dictionary = torch.load(bump_amount_dictionary_path)
else:
    bump_amount_dictionary = {}

    
# Store histogram information
bumps_preds_metrics_path = os.path.join(
    "saved/edit/experiments/bump_edits",
    "bumps_preds_metrics",
    "{}_{}_bumps_preds_metrics.pth".format(target_class_name, n_select))
bumps_preds_metrics = torch.load(bumps_preds_metrics_path)

# Create directories and paths
result_root = os.path.join(
        os.path.dirname(os.path.dirname(config.save_dir)), 
        save_timestamp, 
        '{}_{}'.format(target_class_name, n_select))
ensure_dir(result_root)
progress_report_path = os.path.join(result_root, 'progress_reports.txt')
# Create file to store paths
logit_bump_trial_paths_path = os.path.join(result_root, 'trial_paths.txt')
if os.path.isfile(logit_bump_trial_paths_path):
    os.remove(logit_bump_trial_paths_path)
    
# Iterate through all trial paths
for trial_idx, trial_path in enumerate(original_trial_paths):
    # Obtain trial ID and create save directory for logit bump results
    trial_id = trial_path[len(common_path)+1:]
    logit_bump_trial_save_dir = os.path.join(
        result_root,
        'results',
        trial_id,
        'models')
    ensure_dir(logit_bump_trial_save_dir)
    
    # Obtain desired n_target_predictions
    trial_post_edit_metrics_path = os.path.join(
        trial_path,
        'models',
        'post_edit_metrics.pth')
    trial_post_edit_metrics = torch.load(trial_post_edit_metrics_path)
    n_target_predictions = trial_post_edit_metrics['predicted_class_distribution'][target_class_idx]
    # If bump amount is in dictionary, obtain that value
    if n_target_predictions in bump_amount_dictionary:
        
        informal_log("[{}] Found corresponding logit bump for {} in dictionary ({}/{})".format(
            datetime.now().strftime(r'%m%d_%H%M%S'),
            trial_id,
            trial_idx + 1,
            len(original_trial_paths)), progress_report_path)
        bump_amount = bump_amount_dictionary[n_target_predictions]
        
        log = predict_with_bump(
            data_loader=val_paths_data_loader,
            model=model,
            loss_fn=loss_fn,
            metric_fns=metric_fns,
            device=device,
            target_class_idx=target_class_idx,
            bump_amount=bump_amount,
            output_save_path=os.path.join(logit_bump_trial_save_dir, "post_edit_logits.pth"),
            log_save_path=os.path.join(logit_bump_trial_save_dir, "post_edit_metrics.pth"))
        # Sanity check that the n_target_predictions with bump is within buffer of target
        bump_n_target_predictions = log['predicted_class_distribution'][target_class_idx]
        assert abs(n_target_predictions - bump_n_target_predictions) <= cushion
        
        
    else:
        informal_log("[{}] Creating corresponding logit bump for {} ({}/{})".format(
            datetime.now().strftime(r'%m%d_%H%M%S'),
            trial_id,
            trial_idx + 1,
            len(original_trial_paths)), progress_report_path)
        # Find corresponding bump amount
        bump_amount, n_target_predictions, metrics = \
            match_bump(
                n_target_predictions=n_target_predictions,
                target_class_idx=target_class_idx,
                bumps_preds_metrics=bumps_preds_metrics,
                results_save_dir=logit_bump_trial_save_dir,
                data_loader=val_paths_data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                cushion=cushion,
                debug=True)
        bump_amount_dictionary[n_target_predictions] = bump_amount

    # Write trial path to list
    informal_log(os.path.dirname(logit_bump_trial_save_dir), logit_bump_trial_paths_path)

# Update the bump dictionary on file
ensure_dir(os.path.dirname(bump_amount_dictionary_path))
torch.save(bump_amount_dictionary, bump_amount_dictionary_path)

[0214_113656] Found corresponding logit bump for airplane-train-n03365231_4635/felzenszwalb_masked_softmax in dictionary (1/158)


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:10<00:00, 26.21it/s]


saved/edit/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0213_143741/airplane_100/results/airplane-train-n03365231_4635/felzenszwalb_masked_softmax
[0214_113706] Found corresponding logit bump for airplane-train-n04160586_14463/felzenszwalb_masked_softmax in dictionary (2/158)


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:10<00:00, 25.93it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


saved/edit/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0213_143741/airplane_100/results/airplane-train-n04160586_14463/felzenszwalb_masked_softmax
[0214_113717] Creating corresponding logit bump for airplane-train-n04160586_14463/felzenszwalb_gaussian_softmax (3/158)
target n_predictions: 23921
Initial bounds for bump: (4.416467191958873, 4.547473508864641)


...

100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:09<00:00, 29.71it/s]


cur_bump_amount: 3.3637093110883143, cur_n_target_predictions: 16731
Updated lower bound to 3.3637093110883143


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:09<00:00, 27.98it/s]


cur_bump_amount: 3.370104195710155, cur_n_target_predictions: 16758
Updated upper bound to 3.370104195710155
final results: bump amount: 3.370104195710155 n_target_predictions: 16758
saved/edit/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0213_143741/airplane_100/results/airplane-train-n02704645_17657/felzenszwalb_masked_softmax
[0214_144645] Creating corresponding logit bump for airplane-train-n02704645_17657/felzenszwalb_gaussian_softmax (153/158)


AssertionError: n_target_predictions (8733) must be greater than pre-edit n_target_predictions (9027)

In [18]:
ensure_dir(os.path.dirname(bump_amount_dictionary_path))
torch.save(bump_amount_dictionary, bump_amount_dictionary_path)

### Replicate bump results on a coarse bin level (repeat bins)

In [41]:
# def replicate_bump_results(bumps_preds_metrics,
#                            target_class_distribution,
#                            results_save_dir,
#                            trial_paths_path):
#     '''
#     Given how much to bump logit by to match target class distribution, create results for each tiral
    
#     Arg(s):
#         bumps_preds_metrics : dict
#             result of match_bump_edits()
#         target_class_distribution : dict
#             histogram data, result from save_class_distribution()
#         results_save_dir : str
#             location of results folder
#         trial_paths_path : str
#             path to file to store paths to each trial
            
#     '''
#     pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    
#     metrics = bumps_preds_metrics['metrics']
#     bin_values = target_class_distribution['histogram_bin_values']
#     bins = target_class_distribution['histogram_bins']
#     bin_lows = bins[:-1]
#     bin_highs = bins[1:]

#     trial_paths = []
#     for bin_idx, bin_value in enumerate(tqdm(bin_values)):
#         n_repeat = int(bin_value)
#         if n_repeat == 0:
#             continue
            
#         # Obtain post edit metrics
#         post_edit_metrics = metrics[bin_idx]
        
#         for itr in range(n_repeat):
#             # Create directory for this trial
#             trial_id = 'bin_{}_{}_itr_{}'.format(bin_lows[bin_idx], bin_highs[bin_idx], itr)
#             trial_dir = os.path.join(results_save_dir, trial_id)
            
#             # Log path to trial_paths.txt
#             trial_paths.append(trial_dir)
            
#             trial_save_dir = os.path.join(trial_dir, 'models')
#             ensure_dir(trial_save_dir)
#             # Save pre edit metrics
#             torch.save(pre_edit_metrics, os.path.join(trial_save_dir, 'pre_edit_metrics.pth'))
#             # Save post edit metrics
#             torch.save(post_edit_metrics, os.path.join(trial_save_dir, 'post_edit_metrics.pth'))
        
#     return trial_paths

# def replicate_bump_noise_results(bumps_preds_metrics,
#                                  target_class_distribution,
#                                  results_save_dir,
#                                  trial_paths_path,
#                                  progress_report_path,
#                                  # noise_edit=False,
#                                  target_class_idx,
#                                  config,
#                                  data_loader,
#                                  loss_fn,
#                                  metrics_fns,
#                                  editor,
#                                  device):
#     '''
#     Given how much to bump logit by to match target class distribution, create results for each tiral
    
#     Arg(s):
#         bumps_preds_metrics : dict
#             result of match_bump_edits()
#         target_class_distribution : dict
#             histogram data, result from save_class_distribution()
#         results_save_dir : str
#             location of results folder
#         trial_paths_path : str
#             path to file to store paths to each trial
#         progress_report_path : str
#             path to log progress
#         target_class_idx : int
#             index of target class
#         config : return from ConfigParser
#             config object to instantiate model from
#         data_loader : torch.utils.data.DataLoader
#             validation data loader to obtain metrics for
#         loss_fn : module
#             loss function
#         metric_fns : list[model.metric modules]
#             list of metric functions
#         editor : Editor (from trainer/editor.py) or None
#             pass in an editor if performing random edits
#         device : torch.device
#             GPU device to run model on
#     '''
#     pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    
#     bump_amounts = bumps_preds_metrics['bump_amounts']
#     metrics = bumps_preds_metrics['metrics']
#     bin_values = target_class_distribution['histogram_bin_values']
#     bins = target_class_distribution['histogram_bins']
#     bin_lows = bins[:-1]
#     bin_highs = bins[1:]

#     trial_paths = []
    
#     layernum = config.config['layernum']
#     # if noise_edit:
#     #     assert target_class_idx is not None
#     #     assert config is not None
#     #     assert data_loader is not None
#     #     assert loss_fn is not None
#     #     assert metrics_fns is not None
#     #     assert editor is not None
#     #     assert device is not None
        
#     for bin_idx, bin_value in enumerate(bin_values):
#         n_repeat = int(bin_value)
#         if n_repeat == 0:
#             continue
            
#         informal_log("Bin {} [{}, {})".format(bin_idx, bin_lows[bin_idx], bin_highs[bin_idx]), progress_report_path)
#         for itr in range(n_repeat):
#             informal_log("\tItr {}/{}".format(itr+1, n_repeat), progress_report_path)
#             # Create directory for this trial
#             trial_id = 'bin_{}_{}_itr_{}'.format(bin_lows[bin_idx], bin_highs[bin_idx], itr)
#             trial_dir = os.path.join(results_save_dir, trial_id)

#             # Log path to trial_paths.txt
#             trial_paths.append(trial_dir)
#             informal_log(trial_dir, trial_paths_path)
#             # Create save directories, save pre_edit_metrics
#             trial_save_dir = os.path.join(trial_dir, 'models')
#             pre_edit_metrics_save_path = os.path.join(trial_save_dir, 'pre_edit_metrics.pth')
#             post_edit_metrics_save_path = os.path.join(trial_save_dir, 'post_edit_metrics.pth')
#             ensure_dir(trial_save_dir)
#             # Save pre edit metrics
#             torch.save(pre_edit_metrics, pre_edit_metrics_save_path)
#             if not noise_edit:
#                 # Save calculated post edit metrics
#                 # Obtain and save post edit metrics
#                 post_edit_metrics = metrics[bin_idx]
#                 torch.save(post_edit_metrics, post_edit_metrics_save_path)
                
#             else:
#                 # Load in fresh model
                
#                 model = config.init_obj('arch', module_arch, layernum=layernum, device=device)
#                 model.eval()
                
#                 # edit model with noise
#                 editor.noise_edit(model=model)
                
#                 # obtain bump amount
#                 bump_amount = bump_amounts[bin_idx]
#                 log = predict_with_bump(
#                     data_loader=data_loader,
#                     model=model,
#                     loss_fn=loss_fn,
#                     metric_fns=metric_fns,
#                     device=device,
#                     target_class_idx=target_class_idx,
#                     bump_amount=bump_amount,
#                     save_path=post_edit_metrics_save_path)
            
                
                
#     return trial_paths

In [45]:
# bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
# ensure_dir(bump_save_dir)

# bump_preds_metrics_path = os.path.join(bump_save_dir, 'bumps_preds_metrics.pth')
# target_class_distribution_path = os.path.join(bump_save_dir, 'target_class_distribution.pth')

# # Load objects
# bumps_preds_metrics = torch.load(bump_preds_metrics_path)
# target_class_distribution = torch.load(target_class_distribution_path)

# # Determine whether to use noise_edits or not
# noise_edit = config.config['editor']['noise_edit']
# if noise_edit: 
#     results_save_dir = os.path.join(bump_save_dir, 'bump_noise', 'results')
#     ensure_dir(results_save_dir)
#     progress_report_path = os.path.join(bump_save_dir, 'bump_noise', 'progress_report.txt')
#     trial_paths_path = os.path.join(bump_save_dir, 'bump_noise', 'trial_paths.txt')
# else:
#     results_save_dir = os.path.join(bump_save_dir, 'results')
#     ensure_dir(results_save_dir)
#     progress_report_path = os.path.join(bump_save_dir, 'progress_report.txt')
#     trial_paths_path = os.path.join(bump_save_dir, 'trial_paths.txt')

# print("Performing bump{} edits for {} class".format("-noise" if noise_edit else "", target_class_name))
# print("Trial paths stored at {}".format(trial_paths_path))
# print("Results will be saved to {}".format(results_save_dir))
# # Remove file at trial_paths_path if it exists
# if os.path.isfile(trial_paths_path):
#     os.remove(trial_paths_path)

# print(noise_edit)
# if not noise_edit:
#     trial_paths = replicate_bump_results(
#         bumps_preds_metrics=bumps_preds_metrics,
#         target_class_distribution=target_class_distribution,
#         results_save_dir=results_save_dir,
#         trial_paths_path=trial_paths_path)
# else:
#     # Create editor
#     editor_args = config.config['editor']['args']
#     editor_args['arch'] = config.config['arch']['args']['type']
#     editor = Editor(**editor_args)
#     # Recreate trial results with bump and noise
#     trial_paths = replicate_bump_noise_results(
#         bumps_preds_metrics=bumps_preds_metrics,
#         target_class_distribution=target_class_distribution,
#         results_save_dir=results_save_dir,
#         trial_paths_path=trial_paths_path,
#         progress_report_path=progress_report_path,
#         # Parameters for adding noise
#         # noise_edit=noise_edit,
#         target_class_idx=target_class_idx,
#         config=config,
#         data_loader=val_paths_data_loader,
#         loss_fn=loss_fn,
#         metrics_fns=metric_fns,
#         editor=editor,
#         device=device)

        
    
# print(trial_paths)
# write_lists(trial_paths_path, trial_paths)
# print("Wrote list of trial_paths to {}".format(trial_paths_path))

Performing bump-noise edits for airplane class
Trial paths stored at saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/trial_paths.txt
Results will be saved to saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results
True
Bin 0 [7014.0, 7533.62)
	Itr 1/3
L2 norm of weight change: 4.605295658111572
pre_edit_weights id: 139767148232848 mean: 1.063853324012598e-05
post_edit_weights id: 139767144758304 mean: 1.1618755706876982e-05
are the tensors equal: False


100%|█████████████████████████████████████████████████████████████████████| 274/274 [05:06<00:00,  1.12s/it]


	Itr 2/3
L2 norm of weight change: 4.608025550842285
pre_edit_weights id: 139767148688864 mean: 1.063853324012598e-05
post_edit_weights id: 139767144073552 mean: 8.5252804638003e-06
are the tensors equal: False


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:09<00:00, 29.02it/s]


	Itr 3/3
L2 norm of weight change: 4.610196590423584
pre_edit_weights id: 139767230537296 mean: 1.063853324012598e-05
post_edit_weights id: 139767144108496 mean: 1.4323152754514012e-05
are the tensors equal: False


...

100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:09<00:00, 30.25it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Bin 49 [32475.38, 32995.0)
	Itr 1/1
L2 norm of weight change: 4.606288909912109
pre_edit_weights id: 139767232476560 mean: 1.063853324012598e-05
post_edit_weights id: 139767231556560 mean: 1.4464273590419907e-05
are the tensors equal: False


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:08<00:00, 34.18it/s]

['saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7014.0_7533.62_itr_0', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7014.0_7533.62_itr_1', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7014.0_7533.62_itr_2', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_0', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_1', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_2', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_3', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/b


/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


In [34]:
# # Save class distribution as histogram
# save_class_distribution(
#     csv_path='saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/automobile_100/bump_noise/results_table.csv', 
#     target_class_idx=target_class_idx,
#     target_class_name=target_class_name)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


--- Logging error ---
Traceback (most recent call last):
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/logging/handlers.py", line 69, in emit
    if self.shouldRollover(record):
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/logging/handlers.py", line 186, in shouldRollover
    self.stream.seek(0, 2)  #due to non-posix-compliant Windows feature
OSError: [Errno 116] Stale file handle
Call stack:
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/u/ac4802/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/u/ac4802/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/u/ac4802/.local/lib/python

Saved target class distribution & histogram data to saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/automobile_100/bump_noise/target_class_distribution.pth


{'n_target_predictions': array([ 7828,  7820,  7847,  7778,  7828,  7813,  7824,  8694,  8699,
         8536,  8766,  8791,  8693,  8755,  8695,  8719,  8683,  8655,
         8736,  8644,  9548,  9500,  9308,  9414,  9442,  9516,  9459,
         9490,  9333,  9442,  9494,  9379,  9376,  9303, 10639, 10683,
        10639, 10705, 10822, 10765, 10866, 10790, 10773, 10598, 11074,
        11357, 11458, 11334, 11997, 11706, 13292, 14293, 14461, 14302,
        14142, 13842, 14053, 14068, 14090, 14571, 14658, 15060, 14843,
        14808, 14517, 14717, 14787, 14841, 16025, 15994, 15976, 15963,
        15965, 15987, 16293, 15814, 16077, 16264, 16253, 16328, 15907,
        16148, 16228, 16013, 16183, 16120, 17448, 16606, 16720, 16972,
        17111, 16866, 16381, 16661, 16841, 17077, 17237, 17583, 17898,
        17778, 17948, 17969, 17596, 17670, 17580, 18727, 18502, 18559,
        19167, 19248, 19225, 20326, 22684, 23171, 24276, 23299, 23760,
        25793, 28990, 28034, 31953, 53699]),
 'histog